In [ ]:
import pathlib
import requests
import urllib
import dotenv
import os
import shapely
import shapely.geometry
import geopandas

In [ ]:
SCHEME = "https"
NETLOC_API = "lris.scinfo.org.nz"
WFS_PATH_API_START = "/services;key="
WFS_PATH_API_END = "/wfs"

dotenv.load_dotenv()
KEY = os.environ.get('LRIS_API', None)

CRS = "EPSG:2193"

data_url = urllib.parse.urlunparse((SCHEME, NETLOC_API, f"{WFS_PATH_API_START}{KEY}{WFS_PATH_API_END}", "", "", ""))

In [ ]:
KEY

In [ ]:
x0 = 1752000
x1 = 1753000
y0 = 5430000
y1 = 5440000

In [ ]:
bbox = shapely.geometry.Polygon([(x0, y0), (x0, y1), (x1, y1), (x1, y0)])
bbox = geopandas.GeoSeries([bbox])
bbox = bbox.set_crs(CRS)

In [ ]:
def make_api_params(layer, geometry_type):
    api_query = {
                "service": "WFS",
                "version": 2.0,
                "request": "GetFeature",
                "typeNames": f"layer-{layer}",
                "outputFormat": "json",
                "SRSName": f"{CRS}",
                "cql_filter": f"bbox({geometry_type}, {y0}, {x0}, " +
                              f"{y1}, {x1}, " +
                              f"'urn:ogc:def:crs:{CRS}')"
            }
    return api_query

In [ ]:
def get_features_in_bounds(json_response, bbox):
    crs = json_response['crs']['properties']['name']

    # Cycle through each feature checking in bounds and getting geometry and properties
    features = {'geometry': []}
    for feature in json_response['features']:

        shapely_geometry = shapely.geometry.shape(feature['geometry'])

        # check intersection of tile and catchment in LINZ CRS
        if bbox.intersects(shapely_geometry).any():

            # Create column headings for each 'properties' option from the first in-bounds vector
            if len(features['geometry']) == 0:
                for key in feature['properties'].keys():
                    features[key] = []  # The empty list to append the property values too

            # Convert any one Polygon MultiPolygon to a straight Polygon then add to the geometries
            if (shapely_geometry.geometryType() == 'MultiPolygon' and len(shapely_geometry) == 1):
                shapely_geometry = shapely_geometry[0]
            features['geometry'].append(shapely_geometry)

            # Add the value of each property in turn
            for key in feature['properties'].keys():
                features[key].append(feature['properties'][key])

    # Convert to a geopandas dataframe
    if len(features) > 0:
        features = geopandas.GeoDataFrame(features, crs=crs)
    else:
        features = None
    return features

## North Island pasture productivity

In [ ]:
layer = 105112
geom_type = "Shape"

params = make_api_params(layer, geom_type)

response = requests.get(data_url, params=params, stream=True)
response.raise_for_status()
json_response=response.json()
requests.Request('POST', data_url, params=params).prepare().url
features = get_features_in_bounds(json_response, bbox)

In [ ]:
[features.geometry.area.sum(), features.geometry.length.sum(), features.columns, 
 features.loc[0].geometry.geometryType(), list(features['uid'][0:5])]

# LCDB v5.0

In [ ]:
layer = 104400
geom_type = "GEOMETRY"
params = make_api_params(layer, geom_type)

response = requests.get(data_url, params=params, stream=True)
response.raise_for_status()
json_response=response.json()
requests.Request('POST', data_url, params=params).prepare().url
features = get_features_in_bounds(json_response, bbox)

In [ ]:
[features.geometry.area.sum(), features.geometry.length.sum(), features.columns, 
 features.loc[0].geometry.geometryType(), list(features['Class_2018'][0:5])]

# Create tests for no boundary filter

In [ ]:
def make_api_params_no_bounds(layer):
    api_query = {
                "service": "WFS",
                "version": 2.0,
                "request": "GetFeature",
                "typeNames": f"layer-{layer}",
                "outputFormat": "json",
                "SRSName": f"{CRS}"
            }
    return api_query

In [ ]:
def get_features_no_bounds(json_response):
    crs = json_response['crs']['properties']['name']

    # Cycle through each feature checking in bounds and getting geometry and properties
    features = {'geometry': []}
    for feature in json_response['features']:

        shapely_geometry = shapely.geometry.shape(feature['geometry'])

        # Create column headings for each 'properties' option from the first in-bounds vector
        if len(features['geometry']) == 0:
            for key in feature['properties'].keys():
                features[key] = []  # The empty list to append the property values too

        # Convert any one Polygon MultiPolygon to a straight Polygon then add to the geometries
        if (shapely_geometry.geometryType() == 'MultiPolygon' and len(shapely_geometry) == 1):
            shapely_geometry = shapely_geometry[0]
        features['geometry'].append(shapely_geometry)

        # Add the value of each property in turn
        for key in feature['properties'].keys():
            features[key].append(feature['properties'][key])

    # Convert to a geopandas dataframe
    if len(features) > 0:
        features = geopandas.GeoDataFrame(features, crs=crs)
    else:
        features = None
    return features

## North Island National Pasture Productivity

In [ ]:
params = make_api_params_no_bounds(48556)
response = requests.get(data_url, params=params, stream=True)
response.raise_for_status()
json_response=response.json()
features = get_features_no_bounds(json_response)

In [ ]:
[features.geometry.area.sum(), features.geometry.length.sum(), features.columns, 
 features.loc[0].geometry.geometryType(), list(features['Area'][0:5])]

## LCDB v5.0

In [ ]:
params = make_api_params_no_bounds(48155)
response = requests.get(data_url, params=params, stream=True)
response.raise_for_status()
json_response=response.json()
features = get_features_no_bounds(json_response)

In [ ]:
[features.geometry.area.sum(), features.geometry.length.sum(), features.columns, 
 features.loc[0].geometry.geometryType(), list(features['DOMSOI'][0:5])]